# This notebook is to preprocess a number of images for nn-net training based on the CSV splits yet it can be skipped as we will have to re run all again on the next notebook.


In [1]:
!git clone https://github.com/XLR8-07/FracAtlas.git
!pip install nibabel

Cloning into 'FracAtlas'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 113 (delta 29), reused 41 (delta 14), pack-reused 47
Receiving objects: 100% (113/113), 44.95 MiB | 28.24 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [ ]:
!unzip /'content/drive/MyDrive/bayena/FracAtlas.zip' -d '/content/seg/' > /dev/null


In [ ]:
!tar -xvf "/content/drive/MyDrive/bayena/full_res_images_masks.tar"  > /dev/null

In [4]:
# creating the folder paths need for MSD format
import os
os.makedirs('./MSD/Dataset100_FracAtles/imagesTr')
os.makedirs('./MSD/Dataset100_FracAtles/imagesTs')
os.makedirs('./MSD/Dataset100_FracAtles/labelsTr')


In [5]:
##Data preparetion
import pandas as pd
import cv2
import numpy as np
import nibabel as nib

data_csv = pd.read_csv('/content/seg/FracAtlas/Utilities/Fracture Split/train.csv')
images_folder_path = './content/image_masks/images'
masks_folder_path =   './content/image_masks/masks'
image_size = 224
mode = 'Train'



if mode =='Train':
  #Read and resize the images to 224
  for img_id in data_csv['image_id']:

    img = cv2.imread(images_folder_path +'/'+ img_id )
    img = cv2.resize(img, dsize=(image_size, image_size))

    mask_labels = cv2.imread(masks_folder_path +'/'+ img_id )
    mask_labels = cv2.resize(mask_labels, dsize=(image_size, image_size))

  # Converting to nii.gz
    affine = np.eye(4)
    img_converted_array = np.array(img, dtype=np.float32) # You need to replace normal array by yours
    mask_labels_converted_array = np.array(mask_labels, dtype=np.float32) # You need to replace normal array by yours

    img_nifti = nib.Nifti1Image(img_converted_array, affine)
    mask_labels_nifti = nib.Nifti1Image(mask_labels_converted_array, affine)

  #savin the image and the mask in nii.gz format

    nib.save(img_nifti,'/content/MSD/Dataset100_FracAtles/imagesTr/FRAC_' + img_id[6:10]+'_0000.nii.gz') # Here you put the path + the extionsion 'nii' or 'nii.gz'
    nib.save(mask_labels_nifti,'/content/MSD/Dataset100_FracAtles/labelsTr/FRAC_' + img_id[6:10]+'.nii.gz') # Here you put the path + the extionsion 'nii' or 'nii.gz'







In [8]:

from collections import OrderedDict
import json


task_folder_name = '/content/MSD/Dataset100_FracAtles'
train_label_dir = '/content/MSD/Dataset100_FracAtles/imagesTr'
dataset_name = 'Dataset100_FracAtles'



overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Dataset_folder
json_file_exist = False

if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:

    json_dict = OrderedDict()
    json_dict['name'] = dataset_name
    json_dict['description'] = "FracAtlas: A Dataset for Fracture Classification, Localization and Segmentation of Musculoskeletal Radiographs"
    json_dict['tensorImageSize'] = "3D"
    json_dict['reference'] = "see challenge website"
    json_dict['licence'] = "see challenge website"
    json_dict['release'] = "0.0"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "CT"
    }

    # set expected file ending
    json_dict["file_ending"] = ".nii.gz"

    #label names should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "Fracture": 1,
    }

    train_ids = os.listdir(train_label_dir)
    # test_ids = os.listdir(test_dir)
    json_dict['numTraining'] = len(train_ids)
    # json_dict['numTest'] = len(test_ids)


    with open(os.path.join(task_folder_name,"dataset.json"), 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
        if json_file_exist==False:
            print('dataset.json created!')
        else:
            print('dataset.json overwritten!')

dataset.json created!
